# Puppeteer Startup
Testes dinâmicos do Puppeteer pelo C#

## Instalação

In [3]:
#r "nuget: PuppeteerSharp, 20.2.5"

Installed Packages PuppeteerSharp, 20.2.5

In [11]:
var browserFetcher = new BrowserFetcher();
var installedBrowser = await browserFetcher.DownloadAsync();

installedBrowser

Browser,Chrome
BuildId,138.0.7204.101
Platform,Win64
PermissionsFixed,True


## Matar Processos
Para não ter que fazer isso manualmente um de cada vez ou em caso de processos zumbis

In [ ]:
using System.Diagnostics;

// Use com cuidado
void MatarProcessos(string nomeProcesso)
{
    var processos = Process.GetProcessesByName(nomeProcesso);
    foreach (var p in processos)
    {
        try 
        {
            p.Kill();
            Console.WriteLine($"Matou: {p.ProcessName} (ID: {p.Id})");
        }
        catch (Exception ex)
        {
            // Ignora erros de permissão ou se já morreu
        }
    }
}

// Fecha todos os chromes
MatarProcessos("chrome");

## Imports

In [9]:
// Sistema
using System.IO;
using System.Threading;
using System.Text.Json;
using System.Collections.Generic;

// Puppeteer
using PuppeteerSharp;

## Extensões
Simplificar algumas coisas

In [22]:
public static async Task<string> GetTextAsync(this IElementHandle parentElement, string selector)
{
    var childElement = await parentElement.QuerySelectorAsync(selector);
    if (childElement == null) return string.Empty;

    return await childElement.EvaluateFunctionAsync<string>("node => node.innerText");
}

public static async Task<string> GetSrcImageAsync(this IElementHandle parentElement, string selector)
{
    var childElement = await parentElement.QuerySelectorAsync(selector);
    if (childElement == null) return null;

    return await childElement.EvaluateFunctionAsync<string>("node => node.src");
}

// Seu método de JSON estava correto, pode manter
public static string ToJsonString(this object obj, JsonSerializerOptions options = null)
    => JsonSerializer.Serialize(obj, options ?? new() { WriteIndented = true });

## Variáveis Globais

In [10]:
string tmpDir = Path.GetTempPath();
string userDataDir = Path.Combine(tmpDir, "tmp_chrome");

const int WAIT_TIMEOUT_SECONDS = 15;

const bool SHOW_CHROME = true;
const bool USE_FULL_SCREEN = true;

const bool USE_SERVER = false;
const bool USE_UNSAFE = false;

const bool DISABLE_EXTENSIONS = false;

## Inicialização do Chrome Driver

In [13]:
var puppeteerArgs = new List<string>();
var ignoreDefaultArgs = new List<string>();

// 2. Estabilidade do Sistema & Compatibilidade (Docker/Server/Linux)
if (USE_SERVER)
{
    puppeteerArgs.Add("--no-sandbox"); // Essencial para rodar como root ou em alguns servers
    puppeteerArgs.Add("--disable-dev-shm-usage"); // Evita crash de memória compartilhada em containers
    puppeteerArgs.Add("--disable-gpu"); // Previne erros de renderização gráfica (comum em servidores)
    puppeteerArgs.Add("--remote-allow-origins=*"); // Mantido para compatibilidade, embora o Puppeteer gerencie bem a conexão
}

// 3. Rede e Certificados
if (USE_UNSAFE)
{
    puppeteerArgs.Add("--ignore-certificate-errors"); // Ignora SSL vencido ou auto-assinado
    puppeteerArgs.Add("--ignore-ssl-errors=yes");
}

if (DISABLE_EXTENSIONS)
{
    puppeteerArgs.Add("--disable-extensions"); // A menos que você precise de uma extensão específica
}

// 4. Layout e Visualização
// O Puppeteer define um Viewport de 800x600 por padrão. Precisamos anular isso para usar os argumentos da janela.
ViewPortOptions defaultViewport = null; 

if (USE_FULL_SCREEN)
{
    puppeteerArgs.Add("--start-maximized");
    defaultViewport = null; // NULL é crucial aqui: diz ao Puppeteer para não redimensionar a página e aceitar o tamanho da janela
}
else
{
    puppeteerArgs.Add("--window-size=1920,1080");
    // No Puppeteer é boa prática definir o viewport igual ao window-size se não for maximizado
    defaultViewport = new ViewPortOptions { Width = 1920, Height = 1080 }; 
}

// 5. Limpeza e Performance
puppeteerArgs.Add("--no-first-run");
puppeteerArgs.Add("--no-default-browser-check");
puppeteerArgs.Add("--disable-default-apps");
puppeteerArgs.Add("--disable-notifications"); // Bloqueia "Deseja receber notificações?"
puppeteerArgs.Add("--disable-popup-blocking"); // Permite popups (PJE usa muito para abrir PDFs)

// 6. (Opcional) "Disfarce" Básico
// Remove a barra amarela "Chrome está sendo controlado por software de teste"
// No Puppeteer, isso é feito ignorando o argumento padrão "--enable-automation"
ignoreDefaultArgs.Add("--enable-automation");

// Configuração do Objeto de Lançamento (LaunchOptions)
var options = new LaunchOptions
{
    Headless = !SHOW_CHROME,
    UserDataDir = userDataDir,
    Args = puppeteerArgs.ToArray(),
    DefaultViewport = defaultViewport, // Importante para a lógica de tamanho de tela funcionar
    IgnoreDefaultArgs = true,
    ExecutablePath = browserFetcher.GetExecutablePath(installedBrowser.BuildId) 
};

IBrowser browser = await Puppeteer.LaunchAsync(options);
IPage[] pages = await browser.PagesAsync();
IPage page = pages.First();

page.DefaultTimeout = (int)TimeSpan.FromSeconds(WAIT_TIMEOUT_SECONDS).TotalMilliseconds;
page.DefaultNavigationTimeout = (int)TimeSpan.FromSeconds(WAIT_TIMEOUT_SECONDS).TotalMilliseconds;

In [15]:
await page.GoToAsync("https://www.saucedemo.com");

In [16]:
await page.TypeAsync("#user-name", "standard_user");
await page.TypeAsync("#password", "secret_sauce");
await page.ClickAsync("#login-button");

In [25]:
var elementList = await page.QuerySelectorAllAsync(".inventory_item");

var tasks = elementList.Select(async element =>
{
    var title = await element.GetTextAsync(".inventory_item_name");
    var description = await element.GetTextAsync(".inventory_item_desc");
    var image = await element.GetSrcImageAsync("img.inventory_item_img");
    var priceText = await element.GetTextAsync(".inventory_item_price");

    // Retorna o objeto final já montado
    return new 
    {
        title = title,
        description = description,
        image = image,
        price = double.Parse(priceText.Replace("$", "").Trim())
    };
});

var items = await Task.WhenAll(tasks);

In [26]:
string json = items.ToJsonString();
int quantidade = items.Length;

Console.WriteLine($"Quantidade: {quantidade}");
Console.WriteLine($"Resultado: {json}");

Quantidade: 6
Resultado: [
  {
    "title": "Sauce Labs Backpack",
    "description": "carry.allTheThings() with the sleek, streamlined Sly Pack that melds uncompromising style with unequaled laptop and tablet protection.",
    "image": "https://www.saucedemo.com/static/media/sauce-backpack-1200x1500.0a0b85a385945026062b.jpg",
    "price": 29.99
  },
  {
    "title": "Sauce Labs Bike Light",
    "description": "A red light isn\u0027t the desired state in testing but it sure helps when riding your bike at night. Water-resistant with 3 lighting modes, 1 AAA battery included.",
    "image": "https://www.saucedemo.com/static/media/bike-light-1200x1500.37c843b09a7d77409d63.jpg",
    "price": 9.99
  },
  {
    "title": "Sauce Labs Bolt T-Shirt",
    "description": "Get your testing superhero on with the Sauce Labs bolt T-shirt. From American Apparel, 100% ringspun combed cotton, heather gray with red bolt.",
    "image": "https://www.saucedemo.com/static/media/bolt-shirt-1200x1500.c2599ac5f0